In [1]:
import os
import sys
sys.path.append( os.path.abspath("../../workspace/python-work/python 3/cs230_deeplearning_implement/ec_code/")  )
from class5.week2.Emojify.emo_utils import *

import numpy as np
import emoji
import matplotlib.pyplot as plt

%matplotlib inline

# Emoji表情生成器
# 比如写下“Congratulations on the promotion! Lets get coffee and talk. Love you!”
# ，那么你的表情生成器就会自动生成
# “ Congratulations on the promotion! 👍 Lets get coffee and talk. ☕️ Love you! ❤️”


#1 - 基准模型：Emojifier-V1     
#    对于每个sentence，求出多个word的avg(50,1)，求出对应的w（5,50）  根据y(5,m)训练
#       Emojifier-V1是有缺陷的，比如它不会把“you are not happy”划分为不好一类，
#=======因为它只是将所有单词的向量做了平均，没有关心过顺序
#   1.1 数据集
#   1.2 - Emojifier-V1的结构
#   1.3 - 实现Emojifier-V1
#   1.4 - 验证测试集


# 2 - Emojifier-V2：在Keras中使用LSTM模块
#   2.1 - 模型预览
#   2.2 - Keras与mini-batching
#   2.3 - 嵌入层（ The Embedding layer）
#   4.4 - 构建Emojifier-V2


In [2]:
X_train, Y_train = read_csv(r'C:\workspace\python-work\python 3\cs230_deeplearning_implement\ec_code\class5\week2\Emojify\data\train_emoji.csv')
X_test, Y_test = read_csv(r'C:\workspace\python-work\python 3\cs230_deeplearning_implement\ec_code\class5\week2\Emojify\data\tesss.csv')
maxLen = len(   max(X_train, key=len).split()    )
print(maxLen)
print(max(X_train,key=len))

10
I am so impressed by your dedication to this project


In [3]:
index = 3
print(X_train[index], label_to_emoji(Y_train[index]))


print(type(Y_train))
print("length=",len(set(Y_train)))
y_train_set=set(Y_train)

for y in y_train_set:
    print(y,"==>",label_to_emoji(y))


Miss you so much ❤️
<class 'numpy.ndarray'>
length= 5
0 ==> ❤️
1 ==> ⚾
2 ==> 😄
3 ==> 😞
4 ==> 🍴


In [4]:
#Emojifier-V1的结构
#模型的输入是一段文字（比如“l lov you”），输出的是维度为(1,5)的向量，
# 最后在argmax层找寻最大可能性的输出。


#现在将标签Y转换成softmax分类器所需要的格式，即从(m,1)转换为独热编码(m,5)，
# 每一行都是经过编码后的样本， 其中Y_oh指的是“Y-one-hot”。
Y_oh_train = convert_to_one_hot(Y_train, C = 5)
Y_oh_test = convert_to_one_hot(Y_test, C = 5)

index = 2
print(Y_train[index], "is converted into one hot", Y_oh_train[index])

3 is converted into one hot [0. 0. 0. 1. 0.]


In [5]:
word_to_index, index_to_word, word_to_vec_map =read_glove_vecs(r"C:\workspace\python-work\python 3\cs230_deeplearning_implement\ec_code\class5\week2\Emojify\data\glove6B50d.txt")
#word_to_index：字典类型的词汇（400,001个）与索引的映射（有效范围：0-400,000）
# index_to_word：字典类型的索引与词汇之间的映射。
# word_to_vec_map：字典类型的词汇与对应GloVe向量的映射。

word = "cucumber"
index = 113317
print("the index of", word, "in the vocabulary is", word_to_index[word])
print("the", str(index) + "th word in the vocabulary is", index_to_word[index])


Epoch: 0 --- cost = 1.9520498812810072
Accuracy: 0.3484848484848485


Epoch: 100 --- cost = 0.07971818726014807
Accuracy: 0.9318181818181818


Epoch: 200 --- cost = 0.04456369243681402
Accuracy: 0.9545454545454546


Epoch: 300 --- cost = 0.03432267378786059
Accuracy: 0.9696969696969697


Training set:
Accuracy: 0.9772727272727273
Test set:
Accuracy: 0.8571428571428571


In [6]:
# GRADED FUNCTION: sentence_to_avg
def sentence_to_avg(sentence, word_to_vec_map):
    ''' 
   将句子转换为单词列表，提取其GloVe向量，然后将其平均。
    参数：
        sentence -- 字符串类型，从X中获取的样本。
        word_to_vec_map -- 字典类型，单词映射到50维的向量的字典
        
    返回：
        avg -- 对句子的均值编码，维度为(50,)
    '''    
    # 第一步：分割句子，转换为列表。
    words = sentence.lower().split()
    # 初始化均值词向量
    avg = np.zeros(50,)
    # 第二步：对词向量取平均。
    for w in words:
        avg += word_to_vec_map[w]
    avg = np.divide(avg, len(words))
    return avg




# GRADED FUNCTION: model
def model(X, Y, word_to_vec_map, learning_rate = 0.01, num_iterations = 400):
    """
    在numpy中训练词向量模型。
    
    参数：
        X -- 输入的字符串类型的数据，维度为(m, 1)。
        Y -- 对应的标签，0-7的数组，维度为(m, 1)。
        word_to_vec_map -- 字典类型的单词到50维词向量的映射。
        learning_rate -- 学习率.
        num_iterations -- 迭代次数。
        
    返回：
        pred -- 预测的向量，维度为(m, 1)。
        W -- 权重参数，维度为(n_y, n_h)。
        b -- 偏置参数，维度为(n_y,)
    """
    np.random.seed(1)

    # Define number of training examples
    m = Y.shape[0]                          # number of training examples
    n_y = 5                                 # number of classes  
    n_h = 50                                # dimensions of the GloVe vectors 
    
    # Initialize parameters using Xavier initialization
    W = np.random.randn(n_y, n_h) / np.sqrt(n_h)
    b = np.zeros((n_y,))
    
    # Convert Y to Y_onehot with n_y classes
    Y_oh = convert_to_one_hot(Y, C = n_y) 
    
    # Optimization loop
    for t in range(num_iterations):                       # Loop over the number of iterations
        for i in range(m):                                # Loop over the training examples
            
            # 获取第i个训练样本的均值
            avg = sentence_to_avg(X[i], word_to_vec_map)
            
            # 前向传播
            z = np.dot(W, avg) + b
            a = softmax(z)
            
            # 计算第i个训练的损失
            cost = -np.sum(Y_oh[i]*np.log(a))
            
            # 计算梯度
            dz = a - Y_oh[i]
            dW = np.dot(dz.reshape(n_y,1), avg.reshape(1, n_h))
            db = dz
            
            # 更新参数
            W = W - learning_rate * dW
            b = b - learning_rate * db
        if t % 100 == 0:
            print("Epoch: " + str(t) + " --- cost = " + str(cost))
            pred = predict(X, Y, W, b, word_to_vec_map)
    return pred, W, b



pred, W, b = model(X_train, Y_train, word_to_vec_map)

print("Training set:")
pred_train = predict(X_train, Y_train, W, b, word_to_vec_map)
print('Test set:')
pred_test = predict(X_test, Y_test, W, b, word_to_vec_map)


Epoch: 0 --- cost = 1.9520498812810072
Accuracy: 0.3484848484848485


Epoch: 100 --- cost = 0.07971818726014807
Accuracy: 0.9318181818181818


Epoch: 200 --- cost = 0.04456369243681402
Accuracy: 0.9545454545454546


Epoch: 300 --- cost = 0.03432267378786059
Accuracy: 0.9696969696969697


Training set:
Accuracy: 0.9772727272727273
Test set:
Accuracy: 0.8571428571428571


In [7]:
X_my_sentences = np.array(["i adore you", "you are happy",  "you are not happy"])
Y_my_labels = np.array([[0], [0],[3]])

pred = predict(X_my_sentences, Y_my_labels , W, b, word_to_vec_map)
print_predictions(X_my_sentences, pred)

Accuracy: 0.6666666666666666

i adore you ❤️
you are happy ❤️
you are not happy ❤️


In [10]:
#Emojifier-V2：在Keras中使用LSTM模块
import numpy as np
np.random.seed(0)
from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
np.random.seed(1)
from keras.initializers import glorot_uniform

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 10)                0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 10, 50)            20000050  
_________________________________________________________________
lstm_3 (LSTM)                (None, 10, 128)           91648     
_________________________________________________________________
dropout_3 (Dropout)          (None, 10, 128)           0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dropout_4 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 645       
__________

In [12]:
#Keras与mini-batching
# 在这个部分中，我们会使用mini-batches来训练Keras模型，
# 但是大部分深度学习框架需要使用相同的长度的文字，这是因为如果你使用3个单词与4个单词的句子，
# 那么转化为向量之后，计算步骤就有所不同（一个是需要3个LSTM，另一个需要4个LSTM），所以我们不可能对这些句子进行同时训练。
# 
# 那么通用的解决方案是使用填充。指定最长句子的长度，然后对其他句子进行填充到相同长度。
# 比如：指定最大的句子的长度为20，我们可以对每个句子使用“0”来填充，直到句子长度为20，
# 因此，句子“I love you”就可以表示为(eI,elove,eyou,0⃗ ,0⃗ ,…,0⃗ )
# 在这个例子中，任何任何一个超过20个单词的句子将被截取，所以一个比较简单的方式就是找到最长句子，获取它的长度，然后指定它的长度为最长句子的长度。
# GRADED FUNCTION: sentences_to_indices
def sentences_to_indices(X, word_to_index, max_len):
    """
    输入的是X（字符串类型的句子的数组），再转化为对应的句子列表，
    输出的是能够让Embedding()函数接受的列表或矩阵（参见图4）。
    
    参数：
        X -- 句子数组，维度为(m, 1)
        word_to_index -- 字典类型的单词到索引的映射
        max_len -- 最大句子的长度，数据集中所有的句子的长度都不会超过它。
        
    返回：
        X_indices -- 对应于X中的单词索引数组，维度为(m, max_len)
    """
    m = X.shape[0]                                   # number of training examples
    # 使用0初始化X_indices
    X_indices = np.zeros((m, max_len))
    for i in range(m):
        # 将第i个居住转化为小写并按单词分开。
        sentences_words = X[i].lower().split()
        # 初始化j为0
        j = 0
        # 遍历这个单词列表
        for w in sentences_words:
            # 将X_indices的第(i, j)号元素为对应的单词索引
            X_indices[i, j] = word_to_index[w]
            j += 1
    return X_indices




# GRADED FUNCTION: pretrained_embedding_layer
def pretrained_embedding_layer(word_to_vec_map, word_to_index):
    """
    创建Keras Embedding()层，加载已经训练好了的50维GloVe向量
    参数：
        word_to_vec_map -- 字典类型的单词与词嵌入的映射
        word_to_index -- 字典类型的单词到词汇表（400,001个单词）的索引的映射。
    返回：
        embedding_layer() -- 训练好了的Keras的实体层=========>相当于得到E,(40000,50)
    """
    vocab_len = len(word_to_index) + 1    # adding 1 to fit Keras embedding (requirement)
    emb_dim = word_to_vec_map["cucumber"].shape[0]      # =50  define dimensionality of your GloVe word vectors 
    
    # 初始化嵌入矩阵
    emb_matrix = np.zeros((vocab_len, emb_dim))
    
    # 将嵌入矩阵的每行的“index”设置为词汇“index”的词向量表示
    for word, index in word_to_index.items():
        emb_matrix[index, :] = word_to_vec_map[word]
    #========================================================
    #========================================================
    #========================================================
    #========================================================
    # 定义Keras的embbeding层
    #在Keras中定义嵌入层，当调用Embedding()的时候需要让这一层的参数不能被训练
    #所以我们可以设置trainable=False。
    embedding_layer = Embedding(vocab_len, emb_dim, trainable=False)
    # 构建embedding层
    embedding_layer.build((None,))
    # 将嵌入层的权重设置为嵌入矩阵。
    embedding_layer.set_weights([emb_matrix])
    return embedding_layer



# GRADED FUNCTION: Emojify_V2
def Emojify_V2(input_shape, word_to_vec_map, word_to_index):
    """
    构建Emojifier-V2
    实现Emojify-V2模型的计算图
    参数：
        input_shape -- 输入的维度，通常是(max_len,)
        word_to_vec_map -- 字典类型的单词与词嵌入的映射。
        word_to_index -- 字典类型的单词到词汇表（400,001个单词）的索引的映射。
    返回：
        model -- Keras模型实体
    """
    # 定义sentence_indices为计算图的输入，维度为(input_shape,)，类型为dtype 'int32'
    # 模型的输入是(m, max_len)，定义在了input_shape中
    sentence_indices = Input(input_shape, dtype='int32')
    
    # 创建embedding层,得到E
    embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)
    
    # 通过嵌入层传播sentence_indices，你会得到嵌入的结果====>(50，m,max_len)
    embeddings = embedding_layer(sentence_indices)
    
    # 通过带有128维隐藏状态的LSTM层传播嵌入  n_a=128
    # 需要注意的是，返回的输出应该是一批序列。
    X = LSTM(128, return_sequences=True)(embeddings)
    # 使用dropout，概率为0.5
    X = Dropout(0.5)(X)
    # 通过另一个128维隐藏状态的LSTM层传播X
    # 注意，返回的输出应该是单个隐藏状态，而不是一组序列===>只有a,没有y
    X = LSTM(128, return_sequences=False)(X)    #(n_a,m)
    # 使用dropout，概率为0.5
    X = Dropout(0.5)(X)
    # 通过softmax激活的Dense层传播X，得到一批5维向量。
    X = Dense(5)(X)  #（5，m）
    # 添加softmax激活
    X = Activation('softmax')(X)     #(1,m)
    
    # 创建模型实体
    model = Model(inputs=sentence_indices, outputs=X)
    return model


model = Emojify_V2(  (maxLen,), word_to_vec_map, word_to_index)
model.summary()
#可以看到有“20,223,927”个参数，其中“20,000,050”个参数没有被训练（这是因为它是词向量）
#剩下的是有“223,877”被训练了的。因为我们的单词表有400,001个单词，
# 所以是400,001∗50=20,000,050 400,001*50=20,000,050400,001∗50=20,000,050个不可训练的参数


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 10)                0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 10, 50)            20000050  
_________________________________________________________________
lstm_3 (LSTM)                (None, 10, 128)           91648     
_________________________________________________________________
dropout_3 (Dropout)          (None, 10, 128)           0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dropout_4 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 645       
__________

In [14]:
#使用categorical_crossentropy 损失, adam 优化器与 [‘accuracy’] 指标。
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
X_train_indices = sentences_to_indices(X_train, word_to_index, maxLen)
Y_train_oh = convert_to_one_hot(Y_train, C = 5)

model.fit(X_train_indices, Y_train_oh, epochs = 50, batch_size = 32, shuffle=True)

32/56 [================>.............] - ETA: 0s

56/56 [==============================] - 0s 892us/step


Test accuracy =  0.8750000085149493


In [16]:
#看在测试集的表现
X_test_indices = sentences_to_indices(X_test, word_to_index, max_len = maxLen)
Y_test_oh = convert_to_one_hot(Y_test, C = 5)
loss, acc = model.evaluate(X_test_indices, Y_test_oh)
print("Test accuracy = ", acc)


32/56 [================>.............] - ETA: 0s

56/56 [==============================] - 0s 892us/step


Test accuracy =  0.8750000085149493


In [19]:
# This code allows you to see the mislabelled examples
#看哪些被分类错误了
C = 5
y_test_oh = np.eye(C)[Y_test.reshape(-1)]
X_test_indices = sentences_to_indices(X_test, word_to_index, maxLen)
pred = model.predict(X_test_indices)
for i in range(len(X_test)):
    x = X_test_indices
    num = np.argmax(pred[i])
    if(num != Y_test[i]):
        print('Expected emoji:'+ label_to_emoji(Y_test[i]) + ' prediction: '+ X_test[i] + label_to_emoji(num).strip())


i like this 😄


In [22]:
# Change the sentence below to see your prediction. Make sure all the words are in the Glove embeddings.  
x_test = np.array(['i like this'])
X_test_indices = sentences_to_indices(x_test, word_to_index, maxLen)
print(x_test[0] +' '+  label_to_emoji(np.argmax(model.predict(X_test_indices))))


i like this 😄
